In [ ]:
%pip install pytube
%pip install youtube_transcript_api
%pip install moviepy
%pip install ffmpeg

In [ ]:
import os
from tempfile import tempdir
from uuid import uuid4
import pytube

def download_youtube_video(url: str):
    out_folder = os.path.join(tempdir, "pyrocury", uuid4().hex)
    os.makedirs(out_folder, exist_ok=False)

    yt = pytube.YouTube(url)
    yt.streams.get_highest_resolution().download(out_folder)

    return os.path.join(out_folder, os.listdir(out_folder)[0])

# download_youtube_video("https://www.youtube.com/watch?v=J38Yq85ZoyY")

In [ ]:
import pytube
from youtube_transcript_api import YouTubeTranscriptApi

def get_youtube_transcript(url: str):
    video_id = pytube.YouTube(url).video_id
    return YouTubeTranscriptApi.get_transcript(video_id)

get_youtube_transcript("https://www.youtube.com/watch?v=lj5GXZaE7qs")

In [ ]:
import numpy as np

def convert_raw_heatmap(raw: list[float]):
    # Invert
    H = 10*np.array(raw)
    maxH = max(H)
    K = (1.0-H)/(1.0-maxH)

    # Smoothen (moving average)
    MAVG_WINDOW = 3
    F = []
    
    sum = 0.0
    for i in range(len(K)):
        sum += K[i]
        if i >= MAVG_WINDOW:
            sum -= K[i-MAVG_WINDOW]
        F.append(sum/min(i+1, MAVG_WINDOW))

    return np.array(F)

convert_raw_heatmap([0.09, 0.09, 0.08, 0.09, 0.09, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.07, 0.08, 0.08, 0.09, 0.08, 0.08, 0.08, 0.07, 0.08, 0.07, 0.08, 0.07, 0.08, 0.07, 0.08, 0.08, 0.07, 0.08, 0.08, 0.08, 0.09])

In [22]:
from moviepy.editor import *

def speeden_video(video_file_path: str, speeds: list[float]) -> str:
    out_file = os.path.join(os.path.dirname(video_file_path), "pyrocury-output.mp4")

    video = VideoFileClip(video_file_path)
    video.write_videofile("hello.mp4")
    output = None

    l = 0.0
    for x in range(len(speeds)):
        r = None
        if x != len(speeds)-1:
            r = l+90.0

        sub_video: VideoClip = video.subclip(l, r)
        sub_video = sub_video.set_fps(video.fps*speeds[x])
        sub_video = sub_video.fx(vfx.speedx, speeds[x])

        if output == None:
            output = sub_video
        else:
            output = concatenate_videoclips([output, sub_video])

    output.write_videofile(out_file)
    return out_file

speeden_video("/tmp/pyrocury/1f4922554fc84bc68adfc0dd5f34525d/Introducing YouTube Shorts.mp4", [0.5])


Moviepy - Building video hello.mp4.
MoviePy - Writing audio in helloTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video hello.mp4



TypeError: must be real number, not NoneType